In [2]:
import pandas as pd
import numpy as np
import glob
from utilities import clean_tweet, get_hashtags
from sklearn.metrics import accuracy_score
import tweetnlp
import pickle

def get_pred(model, tweet):
    ouput = model.predict(tweet)
    return ouput['label'] 

# Load models from tweetnlp
model_hate_speech = tweetnlp.load_model('hate')
model_offensive_language = tweetnlp.load_model('offensive')
model_sentiment = tweetnlp.load_model('sentiment')
model_topic = tweetnlp.load_model('topic_classification', multi_label=False)  # Or `model = tweetnlp.TopicClassification(multi_label=False)`

/home/nblomerus/anaconda3/envs/nlp2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
df_benchmark = pd.read_csv('benchmark_100_tweets.csv')
df_benchmark.head()

,Bank,Tweet_Clean,Sport,Sentiment,Hate_Speech,Offensive
0,standard_bank,cellphones and petrol take heat out of inflati...,n,n,n,n
1,fnb,read fnb ceo michael jordaans open letter in b...,n,n,n,n
2,fnb,i hate absa scum applied to move all my bank a...,n,N,n,y
3,fnb,earnings preview fnb quarterly results will be...,n,n,n,n
4,fnb,my mom and family ba ko fnb stadium they went ...,y,n,n,n


In [4]:
# change the values from n and y to not-sport and sport
df_benchmark['Sport'] = df_benchmark['Sport'].replace(['n', 'y'], ['not-sport', 'sport'])
# change the values in the sentiment column from N, n, p to negative, neutral, positive
df_benchmark['Sentiment'] = df_benchmark['Sentiment'].replace(['N', 'n', 'p'], ['negative', 'neutral', 'positive'])
df_benchmark['Hate_Speech'] = df_benchmark['Hate_Speech'].replace(['n', 'y'], ['non-hate', 'hate'])
df_benchmark['Offensive'] = df_benchmark['Offensive'].replace(['n', 'y'], ['non-offensive', 'offensive'])



In [5]:
df_benchmark.head()

,Bank,Tweet_Clean,Sport,Sentiment,Hate_Speech,Offensive
0,standard_bank,cellphones and petrol take heat out of inflati...,not-sport,neutral,non-hate,non-offensive
1,fnb,read fnb ceo michael jordaans open letter in b...,not-sport,neutral,non-hate,non-offensive
2,fnb,i hate absa scum applied to move all my bank a...,not-sport,negative,non-hate,offensive
3,fnb,earnings preview fnb quarterly results will be...,not-sport,neutral,non-hate,non-offensive
4,fnb,my mom and family ba ko fnb stadium they went ...,sport,neutral,non-hate,non-offensive


In [7]:
df_benchmark['Sentiment_Pred'] = df_benchmark['Tweet_Clean'].apply(lambda x: get_pred(model_sentiment, x))
df_benchmark['Hate_Speech_Pred'] = df_benchmark['Tweet_Clean'].apply(lambda x: get_pred(model_hate_speech, x))
df_benchmark['Offensive_Pred'] = df_benchmark['Tweet_Clean'].apply(lambda x: get_pred(model_offensive_language, x))
df_benchmark['Sport_Pred'] = df_benchmark['Tweet_Clean'].apply(lambda x: get_pred(model_topic, x))


In [8]:
df_benchmark.head()

df_benchmark['Sport_Pred'].unique()
# change the values Sport_pred from 'daily', 'economics', 'entertainment' to 'not-sport'
df_benchmark['Sport_Pred'] = df_benchmark['Sport_Pred'].replace(['business_&_entrepreneurs', 'daily_life', 'pop_culture', 'science_&_technology'], ['not-sport', 'not-sport', 'not-sport', 'not-sport']).replace(['sports_&_gaming'], ['sport'])

In [9]:
df_benchmark.head()

,Bank,Tweet_Clean,Sport,Sentiment,Hate_Speech,Offensive,Sentiment_Pred,Hate_Speech_Pred,Offensive_Pred,Sport_Pred
0,standard_bank,cellphones and petrol take heat out of inflati...,not-sport,neutral,non-hate,non-offensive,neutral,non-hate,non-offensive,not-sport
1,fnb,read fnb ceo michael jordaans open letter in b...,not-sport,neutral,non-hate,non-offensive,neutral,non-hate,non-offensive,not-sport
2,fnb,i hate absa scum applied to move all my bank a...,not-sport,negative,non-hate,offensive,negative,non-hate,offensive,not-sport
3,fnb,earnings preview fnb quarterly results will be...,not-sport,neutral,non-hate,non-offensive,neutral,non-hate,non-offensive,not-sport
4,fnb,my mom and family ba ko fnb stadium they went ...,sport,neutral,non-hate,non-offensive,neutral,non-hate,non-offensive,not-sport


In [10]:
# print the accuracy score for each column
print('Accuracy score for Sentiment: ', accuracy_score(df_benchmark['Sentiment'], df_benchmark['Sentiment_Pred']))
print('Accuracy score for Hate Speech: ', accuracy_score(df_benchmark['Hate_Speech'], df_benchmark['Hate_Speech_Pred']))
print('Accuracy score for Offensive: ', accuracy_score(df_benchmark['Offensive'], df_benchmark['Offensive_Pred']))
print('Accuracy score for Topic: ', accuracy_score(df_benchmark['Sport'], df_benchmark['Sport_Pred']))

Accuracy score for Sentiment:  0.7
Accuracy score for Hate Speech:  0.98
Accuracy score for Offensive:  0.95
Accuracy score for Topic:  0.87


In [11]:
df_benchmark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Bank              100 non-null    object
 1   Tweet_Clean       100 non-null    object
 2   Sport             100 non-null    object
 3   Sentiment         100 non-null    object
 4   Hate_Speech       100 non-null    object
 5   Offensive         100 non-null    object
 6   Sentiment_Pred    100 non-null    object
 7   Hate_Speech_Pred  100 non-null    object
 8   Offensive_Pred    100 non-null    object
 9   Sport_Pred        100 non-null    object
dtypes: object(10)
memory usage: 7.9+ KB
